# MNIST Hyper Parameter Test

First, import modules. This includes<br>numpy for computations<br>pandas for data handling<br>matplotlib for display<br>scikit learn for machine learning

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib
from sklearn.svm import SVC
from tqdm import tqdm

%matplotlib inline

sklearn.datasets.fetch_mldata imports datasets found in scikitlearn's database.

In [2]:
#Import data

mnist = fetch_mldata('MNIST original')

Separate the image and labels. The image comes unrolled into a 784 points (28x28 unrolled).

In [3]:
#Separate data and labels

mnist_data = mnist['data']
mnist_target = mnist['target']

Scaling the data to have inputs between -1 and 1 helps improve the model's accuracy.

In [4]:
#Scale mnist data

scaler = StandardScaler()
scaler.fit(mnist_data)
mnist_data_scaled = scaler.transform(mnist_data)

C:\Users\princ\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


The dataset is split in 80% training data and 20% test data. The dataset is stratified during splitting so the training data is evenly distributed among the target values.

In [5]:
#Stratified split of data set to train and test. Allot 20% for test sey

X_train, X_test, y_train, y_test = train_test_split(mnist_data_scaled, 
                                                    mnist_target, 
                                                    stratify=mnist_target, 
                                                    test_size=0.2,
                                                    random_state = 42)

Run a loop to identify which combination of parameters returns the highest accuracy measure.

In [6]:
#Initiate K Nearest Neighbors model

knn_clf_unif_p1 = KNeighborsClassifier(n_neighbors = 10, weights = "uniform", p=1)
knn_clf_unif_p2 = KNeighborsClassifier(n_neighbors = 10, weights = "uniform", p=2)
knn_clf_dist_p1 = KNeighborsClassifier(n_neighbors = 10, weights = "distance", p=1)
knn_clf_dist_p2 = KNeighborsClassifier(n_neighbors = 10, weights = "distance", p=2)

In [7]:
#Train model

knn_clf_unif_p1.fit(X_train,y_train)
knn_clf_unif_p2.fit(X_train,y_train)
knn_clf_dist_p1.fit(X_train,y_train)
knn_clf_dist_p2.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='distance')

Predict based on models

In [8]:
#Predict 
y_pred_unif_p1=[]
y_pred_unif_p2=[]
y_pred_dist_p1=[]
y_pred_dist_p2=[]
for i in tqdm(range(0,X_test.shape[0])):
    y_pred_unif_p1.append(knn_clf_unif_p1.predict(X_test[i,:].reshape(1,-1)))
    y_pred_unif_p2.append(knn_clf_unif_p2.predict(X_test[i,:].reshape(1,-1)))
    y_pred_dist_p1.append(knn_clf_dist_p1.predict(X_test[i,:].reshape(1,-1)))
    y_pred_dist_p2.append(knn_clf_dist_p2.predict(X_test[i,:].reshape(1,-1)))

100%|██████████████████████████████████████████████████████████████████████████| 14000/14000 [1:08:48<00:00,  3.71it/s]


In [10]:
#Score

acc_unif_p1 = accuracy_score(y_pred_unif_p1, y_test)
acc_unif_p2 = accuracy_score(y_pred_unif_p2, y_test)
acc_dist_p1 = accuracy_score(y_pred_dist_p1, y_test)
acc_dist_p2 = accuracy_score(y_pred_dist_p2, y_test)
print("Uniform manhattan dist: ", acc_unif_p1,"\nUniform euclidean dist: ", acc_unif_p2,
      "\nDistance manhattan dist: ", acc_dist_p1,"\nDistance euclidean dist: ", acc_dist_p2)

Uniform manhattan dist:  0.9555 
Uniform euclidean dist:  0.94 
Distance manhattan dist:  0.9584285714285714 
Distance euclidean dist:  0.9416428571428571


In [14]:
svc_try = SVC(random_state = 42, max_iter = 1000, verbose = True,
              gamma = 0.001, C = 100)

In [15]:
svc_try.fit(X_train,y_train)

[LibSVM]

C:\Users\princ\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=1000, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=True)

In [16]:
y_pred_svc = []
for i in tqdm(range(0,X_test.shape[0])):
    y_pred_svc.append(svc_try.predict(X_test[i,:].reshape(1,-1)))

100%|████████████████████████████████████████████████████████████████████████████| 14000/14000 [03:08<00:00, 74.27it/s]


In [17]:
print("SVC: ", accuracy_score(y_pred_svc,y_test))

SVC:  0.9705


In [24]:
svc_9deg = SVC(random_state = 42, max_iter = 50000, kernel = 'poly',
              gamma = 0.001, C = 100, degree = 9)

In [25]:
svc_9deg.fit(X_train,y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=9, gamma=0.001, kernel='poly',
  max_iter=50000, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [26]:
y_pred_svc9deg = []
for i in tqdm(range(0,X_test.shape[0])):
    y_pred_svc9deg.append(svc_9deg.predict(X_test[i,:].reshape(1,-1)))


  1%|█                                                                             | 195/14000 [00:09<11:07, 20.69it/s]


  3%|██▏                                                                           | 385/14000 [00:18<10:57, 20.70it/s]


  4%|███▏                                                                          | 570/14000 [00:27<11:53, 18.82it/s]


  5%|████                                                                          | 728/14000 [00:35<10:28, 21.12it/s]


  7%|█████▏                                                                        | 932/14000 [00:45<10:39, 20.43it/s]


...

 95%|████████████████████████████████████████████████████████████████████████▎   | 13322/14000 [11:12<00:35, 18.89it/s]


 96%|█████████████████████████████████████████████████████████████████████████▎  | 13498/14000 [11:21<00:26, 19.11it/s]


 98%|██████████████████████████████████████████████████████████████████████████▏ | 13656/14000 [11:29<00:17, 19.67it/s]


 99%|███████████████████████████████████████████████████████████████████████████ | 13831/14000 [11:38<00:08, 19.58it/s]


100%|████████████████████████████████████████████████████████████████████████████| 14000/14000 [11:46<00:00, 19.49it/s]


In [27]:
print("SVC 9 Deg: ", accuracy_score(y_pred_svc9deg,y_test))

SVC 9 Deg:  0.3742142857142857


In [29]:
svc_high = SVC(random_state = 42, max_iter = 100000,
              gamma = 0.001, C = 100, tol = 0.0001)

svc_high.fit(X_train,y_train)

y_pred_svc_high = []
for i in tqdm(range(0,X_test.shape[0])):
    y_pred_svc_high.append(svc_high.predict(X_test[i,:].reshape(1,-1)))



  3%|██▋                                                                           | 481/14000 [00:07<03:20, 67.28it/s]


  7%|█████▍                                                                        | 973/14000 [00:14<03:14, 67.08it/s]


 11%|████████▎                                                                    | 1511/14000 [00:21<02:57, 70.21it/s]


 15%|███████████▏                                                                 | 2044/14000 [00:29<02:50, 70.28it/s]


 18%|██████████████▏                                                              | 2570/14000 [00:37<02:46, 68.80it/s]


...

 90%|████████████████████████████████████████████████████████████████████▍       | 12598/14000 [03:04<00:21, 64.89it/s]


 94%|███████████████████████████████████████████████████████████████████████     | 13095/14000 [03:11<00:13, 66.05it/s]


 97%|█████████████████████████████████████████████████████████████████████████▊  | 13596/14000 [03:19<00:06, 65.53it/s]


100%|████████████████████████████████████████████████████████████████████████████| 14000/14000 [03:25<00:00, 68.27it/s]

NameError: name 'accuracyscore' is not defined

In [30]:
acc_svc_high = accuracy_score(y_pred_svc_high, y_test)
acc_svc_high

0.971